# Timeline

In [ ]:
# please provide your OPENAI_KEY
OPENAI_KEY = "sk-<your-openai-key>"

## Related Functions

In [ ]:
import json
import re
import random
import requests
import base64

def run_local_vision_request(text, image_urls, temperature=0):
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    api_key = OPENAI_KEY

    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": text},
            ]
        }
    ]

    for image_path in image_urls:
        # Getting the base64 string
        base64_image = encode_image(image_path)
        messages[0]["content"].append({
            "image_url": f"data:image/jpeg;base64,{base64_image}"
        })

    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": messages,
        "max_tokens": 2048,
        "temperature": temperature,
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

def run_test(name, description, prompt, image_urls, question, logger, times=1):
    reason_list = []
    score_list = []
    while len(score_list) < times:
        shuffled_list = image_urls.copy()
        random.shuffle(shuffled_list)
        index = [image_urls.index(x) for x in shuffled_list]
        try:
            response = run_local_vision_request(
                text=prompt, 
                image_urls=shuffled_list,
            )
            matched_content = re.search(r'```json([\s\S]*?)```', response["choices"][0]["message"]["content"])
            response_list = json.loads(matched_content.group(1))
            score = sorted(response_list, key=lambda x: index[response_list.index(x)])
            reason_list.append(response["choices"][0]["message"]["content"])
            score_list.append(sorted(response_list, key=lambda x: index[response_list.index(x)]))
            logger.log(name, description, prompt, shuffled_list, response["choices"][0]["message"]["content"], score, question)
        except Exception as e:
            print(e)
    return reason_list

class Logger:
    def __init__(self, file_path):
        self.file_path = file_path
        
    def log(self, name, description, prompt, image_urls, response, score, question):
        json_data_to_add = {"name": name, "description": description, "prompt": prompt, "image_urls": image_urls, "response": response, "score": score, "question": question}

        with open(self.file_path, 'r') as file:
            data = json.load(file)

        data.append(json_data_to_add)

        with open(self.file_path, 'w') as file:
            json.dump(data, file, indent=4)
logger = Logger("./timeline.json")

## Prompt Preparation

In [ ]:
ROLE = "You are an average user. "
WHEN_QUESTION = "There are several different timelines represented on different shapes with same dataset (Horizontal, Vertical line, Circle, Spiral). The data presented on the timeline is made up specifically for this experiment. You are given the question as '{question}', and you need to select one from {number} options: {options}."
WHAT_QUESTION = "The timelines contain different events and be represented on different shapes. The data presented on the timeline is made up specifically for this experiment. You are given the question as '{question}', and you need to select one from {number} options: {options}."
FIND_QUESTION = "The timelines contain different events and be represented on different shapes. The data presented on the timeline is made up specifically for this experiment. You are given the question as '{question}'."
COMPARE_QUESTION = "The timelines contain different events and be represented on different shapes. The data presented on the timeline is made up specifically for this experiment. You are given the question as '{question}', and you need to select one from {number} options: {options}."
EASIEST_READ = "Please complete the above question first and give your answer. Then you are given the question 'Which timeline shape was the easiest to read for the historical dataset?', and you need to select one from 5 options: {options}."
JSON_FORMAT = 'Please give an additional scoring result in json format at the end of your answe, like ```json[{{"answer": answer, "easiest_read": easiest_read}}]```. '

## Data Preparation

In [ ]:
choice_list = ['Horizontal', 'Vertical line', 'Circle', 'Spiral', 'No preference']
image_list = ['schedule.png', 'history.png', 'plants.png']
task_list = ['WHEN', 'WHAT', 'FIND', 'COMPARE']
dataset_list = ['schedule', 'history', 'plants']
file_path = './experiment_material/stimuli/'

questions = [
    {
        "question": "At what time was the Group Meeting on Wednesday?",
        "options": ['8 AM', '12 PM', '2 PM', '4 PM', '7 PM', '8 PM'],
        "answer": '2 PM',
        "dataset": "schedule",
        "dataset_type": "mixed",
        "question_type": "WHEN",
    },
    {
        "question": "In what year was writing invented?",
        "options": ['921','924','925','927','929','933'],
        "answer": '929',
        "dataset": "history",
        "dataset_type": "non-recurrent",
        "question_type": "WHEN",
    },
    {
        "question": "In which season do you plant Puffapod?",
        "options": ['Spring', 'Summer', 'Autumn', 'Winter','Anytime','Not sure'],
        "answer": 'Autumn',
        "dataset": "plants",
        "dataset_type": "recurrent",
        "question_type": "WHEN",
    },
    {
        "question": "What happened on Tuesday at 12 PM?",
        "options": ['Gym', 'Social hour', 'Wake up', 'Reading Group', 'Lunch', 'Group Meeting'],
        "answer": 'Lunch',
        "dataset": "schedule",
        "dataset_type": "mixed",
        "question_type": "WHAT",
    },
    {
        "question": "What happened in 935?",
        "options": ['City is built', 'Temples built', 'Economy booms', 'Bartering ends','Trading flourishes', 'Farming starts'],
        "answer": 'City is built',
        "dataset": "history",
        "dataset_type": "non-recurrent",
        "question_type": "WHAT",
    },
    {
        "question": "What happens in Autumn 2019?",
        "options": ['Planting Wolfsbane', 'Planting Dittany', 'Planting Puffapod', 'Planting Gurdyroot', 'Planting nothing', 'Not sure'],
        "answer": 'Planting Puffapod',
        "dataset": "plants",
        "dataset_type": "recurrent",
        "question_type": "WHAT",
    },
    {
        "question": "Movie Night happened on Tuesday 6 PM. Find it on the timeline and click on it.",
        "options": [],
        "answer": 'Movie Night',
        "dataset": "schedule",
        "dataset_type": "mixed",
        "question_type": "FIND",
    },
    {
        "question": "In the year 912, there was a major epidemic. Find it on the timeline and click on it.",
        "options": [],
        "answer": 'Major epidemic',
        "dataset": "history",
        "dataset_type": "non-recurrent",
        "question_type": "FIND",
    },
    {
        "question": "Wolfsbane is planted in Winter 2018. Find it on the timeline and click on it.",
        "options": [],
        "answer": 'Planting Wolfsbane',
        "dataset": "plants",
        "dataset_type": "recurrent",
        "question_type": "FIND",
    },
    {
        "question": "Is Group Meeting on Wednesday closer to Wake up on Wednesday or Halloween Party on Wednesday?",
        "options": ['Wake up on Wednesday', 'Halloween Party on Wednesday', 'Same', 'Not sure'],
        "answer": 'Halloween Party on Wednesday',
        "dataset": "schedule",
        "dataset_type": "mixed",
        "question_type": "COMPARE",
    },
    {
        "question": "Is 'Bartering ends' closer to 'Farming starts' or 'Writing invented'?",
        "options": ['Farming starts', 'Writing invented', 'Same', 'Not sure'],
        "answer": 'Same',
        "dataset": "history",
        "dataset_type": "non-recurrent",
        "question_type": "COMPARE",
    },
    {
        "question": "In a year, is planting Puffapod closer to planting Dittany or planting Wolfsbane?",
        "options": ['Planting Dittany', 'Planting Wolfsbane', 'Same', 'Not sure'],
        "answer": 'Planting Wolfsbane',
        "dataset": "plants",
        "dataset_type": "recurrent",
        "question_type": "COMPARE",
    },
]

## Task Execution

In [ ]:
for task in task_list:
    name = "timeline_experiment"
    description = task
    for dataset in dataset_list:
        for question in questions:
            if question['question_type'] == task and question['dataset'] == dataset:
                image_urls = [file_path + dataset + '.png']
                shuffled_list = choice_list.copy()
                random.shuffle(shuffled_list)
                prompt = ROLE + \
                        WHEN_QUESTION.format(question=question['question'], number=len(question['options']), options=str(question['options'])) + \
                        EASIEST_READ.format(options=shuffled_list) + \
                        JSON_FORMAT + \
                        "DON'T say sorry or you cannot. YOU CAN."
                run_test(
                    name=name,
                    description=description,
                    prompt=prompt,
                    image_urls=image_urls,
                    logger=logger,
                    question=question,
                    times=5)